# LangGraph Example: LLM with Tools
This Notebook tests out using different LLMs to power a custom LangGraph agent

### Notebook setup

In [67]:
# %pip install --upgrade langchain langsmith langgraph langchain_openai langchain-together

In [68]:
from dotenv import load_dotenv
import os

# Load .env file
load_dotenv()

# Set model variables
OPENAI_BASE_URL = "https://api.openai.com/v1"
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_ORGANIZATION"] = os.getenv("OPENAI_ORGANIZATION")

OLLAMA_BASE_URL = "http://localhost:11434/v1"
OLLAMA_API_KEY = "ollama"

TOGETHER_BASE_URL = "https://api.together.xyz/v1"
TOGETHER_API_KEY = os.getenv("TOGETHER_API_KEY")

# Initialize LangSmith
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = "Demos"

# Initialize OpenWeatherMap
OPENWEATHERMAP_API_KEY = os.getenv("OPENWEATHERMAP_API_KEY")

### Set up the Tools


In [69]:
# Import things that are needed generically for tools
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import StructuredTool

In [70]:
import requests


class City(BaseModel):
    city: str = Field(description="City")
    country: str = Field(description="Country code")


def get_current_weather(city: str, country: str) -> int:
    response = requests.get(
        f"http://api.openweathermap.org/data/2.5/weather?q={city},{country}&appid={OPENWEATHERMAP_API_KEY}"
    )
    data = response.json()
    temp_kelvin = data["main"]["temp"]
    temp_fahrenheit = (temp_kelvin - 273.15) * 9 / 5 + 32
    return int(temp_fahrenheit)


weather = StructuredTool.from_function(
    func=get_current_weather,
    name="Get_Weather",
    description="Get the current temperature from a city, in Fahrenheit",
    args_schema=City,
    return_direct=False,
)

In [71]:
class DifferenceInput(BaseModel):
    minuend: int = Field(
        description="The number from which another number is to be subtracted"
    )
    subtrahend: int = Field(description="The number to be subtracted")


def get_difference(minuend: int, subtrahend: int) -> int:
    return minuend - subtrahend


difference = StructuredTool.from_function(
    func=get_difference,
    name="Calculate_difference",
    description="Calculate the difference between two temperatures in Fahrenheit",
    args_schema=DifferenceInput,
    return_direct=False,
)

In [72]:
tools = [weather, difference]

In [73]:
# Set up the tools to execute them from the graph
from langgraph.prebuilt import ToolExecutor

tool_executor = ToolExecutor(tools)

### Set up the agent


In [74]:
# Define the response schema for our agent
from langchain_core.pydantic_v1 import BaseModel, Field


class Response(BaseModel):
    """Format the final answer to the user's question"""

    warmest_city: str = Field(description="The warmest city and its current temperature")
    explanation: str = Field(
        description="How much warmer it is in the warmest city than the other cities"
    )

In [75]:
from langchain_openai import ChatOpenAI
from langchain_core.utils.function_calling import convert_to_openai_function, convert_to_openai_tool
from langchain_together import Together

from langchain_core.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
)
from langchain_core.runnables import RunnablePassthrough

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant",
        ),
        MessagesPlaceholder(variable_name="messages", optional=True),
    ]
)

# Create the LLM
# OpenAI
# llm = ChatOpenAI(
#     openai_api_base=OPENAI_BASE_URL,
#     api_key=OPENAI_API_KEY,
#     model="gpt-3.5-turbo-1106",
# )

# # Local (Ollama)
# llm = ChatOpenAI(
#     openai_api_base=OLLAMA_BASE_URL,
#     api_key=OLLAMA_API_KEY,
#     model="llama2",
# )

# Serverless (with Together.ai)
llm = ChatOpenAI(
    openai_api_base=TOGETHER_BASE_URL,
    api_key=TOGETHER_API_KEY,
    model="mistralai/Mixtral-8x7B-Instruct-v0.1",
    # model="mistralai/Mistral-7B-Instruct-v0.1",
    # model="togethercomputer/CodeLlama-34b-Instruct",
)

tools_to_pass_in = [convert_to_openai_function(t) for t in tools]
# tools_to_pass_in.append(convert_to_openai_function(Response))

model = {"messages": RunnablePassthrough()} | prompt | llm.bind_tools(tools_to_pass_in)

### Set up the Agent State

Everything from here onwards is new


In [76]:
from typing import TypedDict, Annotated, Sequence
import operator
from langchain_core.messages import BaseMessage


class AgentState(TypedDict):
    messages: Sequence[BaseMessage]

### Set up the node actions


In [77]:
from langgraph.prebuilt import ToolInvocation
import json
from langchain_core.messages import ToolMessage


# Define the function that determines whether to continue or not
def should_continue(state):
    last_message = state["messages"][-1]
    # If there are no tool calls, then we finish
    if "tool_calls" not in last_message.additional_kwargs:
        return "end"
    # If there is a Response tool call, then we finish
    elif any(
        tool_call["function"]["name"] == "Response"
        for tool_call in last_message.additional_kwargs["tool_calls"]
    ):
        return "end"
    # Otherwise, we continue
    else:
        return "continue"


# Define the function that calls the model
def call_model(state):
    messages = state["messages"]
    response = model.invoke(messages)
    return {"messages": messages + [response]}


# Define the function to execute tools
def call_tool(state):
    messages = state["messages"]
    # We know the last message involves at least one tool call
    last_message = messages[-1]

    # We loop through all tool calls and append the message to our message log
    for tool_call in last_message.additional_kwargs["tool_calls"]:
        action = ToolInvocation(
            tool=tool_call["function"]["name"],
            tool_input=json.loads(tool_call["function"]["arguments"]),
            id=tool_call["id"],
        )

        # We call the tool_executor and get back a response
        response = tool_executor.invoke(action)
        # We use the response to create a FunctionMessage
        function_message = ToolMessage(
            content=str(response), name=action.tool, tool_call_id=tool_call["id"]
        )

        # Add the function message to the list
        messages.append(function_message)

    # We return a list, because this will get added to the existing list

    return {"messages": messages}

### Define the Graph


In [78]:
from langgraph.graph import StateGraph, END

# Initialize a new graph
graph = StateGraph(AgentState)

# Define the two Nodes we will cycle between
graph.add_node("agent", call_model)
graph.add_node("action", call_tool)

# Set the Starting Edge
graph.set_entry_point("agent")

# Set our Contitional Edges
graph.add_conditional_edges(
    "agent",
    should_continue,
    {
        "continue": "action",
        "end": END,
    },
)

# Set the Normal Edges
graph.add_edge("action", "agent")

# Compile the workflow
app = graph.compile()

### Run our graph


In [79]:
from langchain_core.messages import HumanMessage

inputs = {
    "messages": [
        HumanMessage(
            content="Where is it warmest: Austin, Texas; Tokyo; or Seattle? And by how much is it warmer than the other cities?"
        )
    ]
}
for output in app.with_config({"run_name": "LLM with Tools"}).stream(inputs):
    # stream() yields dictionaries with output keyed by node name
    for key, value in output.items():
        print(f"Output from node '{key}':")
        print("---")
        print(value)
    print("\n---\n")

Output from node 'agent':
---
{'messages': [HumanMessage(content='Where is it warmest: Austin, Texas; Tokyo; or Seattle? And by how much is it warmer than the other cities?'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_wxcfhv29kmdi9vhy8xsu6b4u', 'function': {'arguments': '{"minuend":78,"subtrahend":68}', 'name': 'Calculate_difference'}, 'type': 'function'}]})]}

---

Output from node 'action':
---
{'messages': [HumanMessage(content='Where is it warmest: Austin, Texas; Tokyo; or Seattle? And by how much is it warmer than the other cities?'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_wxcfhv29kmdi9vhy8xsu6b4u', 'function': {'arguments': '{"minuend":78,"subtrahend":68}', 'name': 'Calculate_difference'}, 'type': 'function'}]}), ToolMessage(content='10', tool_call_id='call_wxcfhv29kmdi9vhy8xsu6b4u', name='Calculate_difference')]}

---

Output from node 'agent':
---
{'messages': [HumanMessage(content='Where is it warmest: Austin, Texas; Tok